1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [1]:
def conversion(num, orig, dest):
    
    #from dec to bin
    if orig == 'deca' and dest == 'binary':
        out = bin(num)
        
    #from dec to hex
    elif orig == 'deca' and dest == 'hexa':
        out = hex(num)        
        
    #from bin to dec
    elif orig == 'binary' and dest == 'deca':
        out = int(num, 2)
    
    #from bin to hex
    elif orig == 'binary' and dest == 'hexa':
        num = int(num, 2)
        out = hex(num)
       
    #from hex to bin
    elif orig == 'hexa' and dest == 'binary':
        num = int(num, 16)
        out = bin(num)
       
    #from hex to dec
    elif orig == 'hexa' and dest == 'deca':
        out = int(num, 16)
        
    return out

In [2]:
origin = 'hexa'
destination = 'binary'
value = '0xa'
converted = conversion(value, origin, destination)
print(f'The {origin} number is {value} and its conversion to {destination} is {converted}')

origin = 'deca'
destination = 'binary'
value = 10
converted = conversion(value, origin, destination)
print(f'The {origin} number is {value} and its conversion to {destination} is {converted}')

origin = 'binary'
destination = 'deca'
value = '0b1010'
converted = conversion(value, origin, destination)
print(f'The {origin} number is {value} and its conversion to {destination} is {converted}')

The hexa number is 0xa and its conversion to binary is 0b1010
The deca number is 10 and its conversion to binary is 0b1010
The binary number is 0b1010 and its conversion to deca is 10


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [3]:
# from the 32 bit to the correspondent number
def conversion_32(word):
    
    
    #sign
    if word[0] == '0':
        sign = 1
    else:
        sign = -1
    
    #exponent
    exp = word[1:9]
    exp = '0b'+exp
    exp = int(exp, 2)
    deca_exp = int(exp) - 127
    
    #mantissa
    mantissa = 0
    for index, val in enumerate(word[9:len(word)]):
        mantissa = mantissa + int(val)*(2**(-(index+1)))
        
    #final values
    output = ((-1)**sign) * (1+round(mantissa,5)) * (2**deca_exp)
    
    #check
    if sign == 1 and exp == 255 and mantissa == 0:
        return float("inf")
    
    elif sign == -1 and exp == 255 and mantissa == 0:
        return float("-inf")
    
    elif exp == 255 and mantissa > 0:
        return float("Nan")

    else:
        return output
        

In [4]:
word = '11000000101100000000000000000001'
conv = conversion_32(word)
print(f'Conversion for {word} in single floating point is {conv}')

word = '01111111100000000000000000000000'
conv = conversion_32(word)
print(f'Conversion for {word} in single floating point is {conv}')

word = '11111111100000000000000000000000'
conv = conversion_32(word)
print(f'Conversion for {word} in single floating point is {conv}')

word = '11111111100000000000000000000001'
conv = conversion_32(word)
print(f'Conversion for {word} in single floating point is {conv}')


Conversion for 11000000101100000000000000000001 in single floating point is -5.5
Conversion for 01111111100000000000000000000000 in single floating point is inf
Conversion for 11111111100000000000000000000000 in single floating point is -inf
Conversion for 11111111100000000000000000000001 in single floating point is nan


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [5]:
def limits():
    underflow = 1.0
    overflow = 1.0
    
    for i in range(10000):
       
        while underflow != 0:
            old_underflow = underflow
            underflow = underflow/2            
            
        while overflow < float('inf'):
            old_overflow = overflow
            overflow = overflow*2
    
    return old_underflow, old_overflow
        

In [6]:
under, over = limits()

print(f'The underflow limit is {under}, while the overflow one is {over}')
print(f'If I divided again the underflow limit the result is {under/2}, while multplying the overflow it is {over*2}')

The underflow limit is 5e-324, while the overflow one is 8.98846567431158e+307
If I divided again the underflow limit the result is 0.0, while multplying the overflow it is inf


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [7]:
def precision():
    
    val = 1.0
    old_val = 0.0
    additive = 0.1
    count = 1
    
    for i in range(200):       
        while old_val < val:
            old_val = val
            val = val + additive/(10**count)
            count = count+1

    return old_val, count

In [8]:
val, count = precision()
print(f'The machine precision is {1/(10**(count-1))}')
print(f'If I add {1/(10**(count))} to {val}, the obtained result is the same: {val+1/(10**(count))}')
print(f'Instead if I add {1/(10**(count-1))} to {val}, the obtained result changes: {val+1/(10**(count-1))}')

The machine precision is 1e-15
If I add 1e-16 to 1.011111111111111, the obtained result is the same: 1.011111111111111
Instead if I add 1e-15 to 1.011111111111111, the obtained result changes: 1.0111111111111122


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [9]:
import math
def quadratic(a,b,c):
    root = math.sqrt(math.pow(b,2) - 4*a*c)
    x_pos = (-b+root)/(2*a)
    x_neg = (-b-root)/(2*a)

    return x_pos, x_neg

a,b,c = 0.001, 1000, 0.001
x_pos, x_neg = quadratic(a,b,c)
print(f'The two solutions of the quadratic form are: {x_pos} and {x_neg}')

The two solutions of the quadratic form are: -9.999894245993346e-07 and -999999.999999


In [10]:
def quadratic_b(a,b,c):
    root = math.sqrt(math.pow(b,2) - 4*a*c)
    x_pos = ((-b+root)*(-b-root)) / ((2*a)*(-b-root))
    x_neg = ((-b-root)*(-b+root)) / ((2*a)*(-b+root))
    
    return x_pos, x_neg

a,b,c = 0.001, 1000, 0.001
x_pos_b, x_neg_b = quadratic_b(a,b,c)
print(f'The two solutions of the quadratic form are: {x_pos_b} and {x_neg_b}')

The two solutions of the quadratic form are: -9.999894245993346e-07 and -999999.9999990001


In [18]:
a,b,c = 0.001, 1000, 0.001
root = math.sqrt(math.pow(b,2) - 4*a*c)

print(f'The two result differs in the negative root')
print(f'In a) the numerator is {-b-root} and the denominator is {2*a}')
print('Both of them have less than 15 significative digits')
print(f'In b) the numerator is {(-b-root)*(-b+root)} while the denominator is {(2*a)*(-b+root)}')
print('Both pf them have 15 significative digits, which is the maximum number of significative digits')
print('Operating with such number at the precision limits could lead to some calculation problems')


The two result differs in the negative root
In a) the numerator is -1999.999999998 and the denominator is 0.002
Both of them have less than 15 significative digits
In b) the numerator is 3.999957698393339e-06 while the denominator is -3.999957698397338e-12
Both pf them have 15 significative digits, which is the maximum number of significative digits
Operating with such number at the precision limits could lead to some calculation problems


In [12]:
def quadratic_accurate(a,b,c, re_exp):
    root = math.sqrt(math.pow(b,2) - 4*a*c)
    x_pos = (-b+root)/(2*a)
    x_neg = (-b-root)/(2*a)
    
    if re_exp == True:
        pos_term = (-b-root)/(-b-root)
        neg_term = (-b+root)/(-b+root)
        x_pos = x_pos * pos_term
        x_neg = x_neg * neg_term
    
    return x_pos, x_neg

a,b,c = 0.001, 1000, 0.001
x_pos_b, x_neg_b = quadratic_accurate(a,b,c, True)
print(f'The two solutions of the quadratic form as in case a) are: {x_pos_b} and {x_neg_b}')

a,b,c = 0.001, 1000, 0.001
x_pos_b, x_neg_b = quadratic_accurate(a,b,c, False)
print(f'The two solutions of the quadratic form as in case b) are: {x_pos_b} and {x_neg_b}')
    

The two solutions of the quadratic form as in case a) are: -9.999894245993346e-07 and -999999.999999
The two solutions of the quadratic form as in case b) are: -9.999894245993346e-07 and -999999.999999


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [20]:
def f(x):
    return x*(x-1)


val = 1
delta = 0.01

der = (f(val+delta) - f(val))/ delta

print(f'The derivative of the function for delta equal to {delta} is {der}')
print(f"The error with the analytical result f'(1) = 1 is {1-der}")
print(f'The problem stays in the number of significative digits in the numerator')

The derivative of the function for delta equal to 0.01 is 1.010000000000001
The error with the analytical result f'(1) = 1 is -0.010000000000000897
The problem stays in the number of significative digits in the numerator


In [21]:
val = 1
for i in range(4,15,2):
    delta = 10**(-i)
    der = (f(val+delta) - f(val))/ delta
    print(f'The derivative of the function for delta equal to {delta} is {der}')
    print(f"The error with the analytical result f'(1) = 1 is {1-der}")
    
print('As delta increase the error decrease up to 1e-8, while then it increase again')
print('For this reason the algorithm is unstable')

The derivative of the function for delta equal to 0.0001 is 1.0000999999998899
The error with the analytical result f'(1) = 1 is -9.999999988985486e-05
The derivative of the function for delta equal to 1e-06 is 1.0000009999177333
The error with the analytical result f'(1) = 1 is -9.99917733279787e-07
The derivative of the function for delta equal to 1e-08 is 1.0000000039225287
The error with the analytical result f'(1) = 1 is -3.922528746258536e-09
The derivative of the function for delta equal to 1e-10 is 1.000000082840371
The error with the analytical result f'(1) = 1 is -8.284037100736441e-08
The derivative of the function for delta equal to 1e-12 is 1.0000889005833413
The error with the analytical result f'(1) = 1 is -8.890058334132256e-05
The derivative of the function for delta equal to 1e-14 is 0.9992007221626509
The error with the analytical result f'(1) = 1 is 0.0007992778373491216
As delta increase the error decrease up to 1e-8, while then it increase again
For this reason th

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [15]:
def f(x):
    return math.sqrt(1-x*x)

def integral(func,N,a,b):
    
    val = 0
    
    for i in range(1, N+1):
        val = val + f(b+((i-(1/2))*((a-b)/N)))
    
    return ((a-b)/N)*val

N = 100
out = integral(f,N,1,-1)
print(f'The integral with N equal to {N} is {out}')
print(f'The difference with the true value is {out - math.pi/2}')

The integral with N equal to 100 is 1.5712827762297954
The difference with the true value is 0.00048644943489883907


In [16]:
import time
tic = time.perf_counter()
out = integral(f,1330000,1,-1)
toc = time.perf_counter()
print(f'The integral with N equal to 1330000 is {out} and takes {toc-tic} s to run')
print(f'The difference with the true value is {out - math.pi/2}')

The integral with N equal to 1330000 is 1.5707963271124443 and takes 1.0012852 s to run
The difference with the true value is 3.1754776586012667e-10


In [17]:
tic = time.perf_counter()
out = integral(f,100000000,1,-1)
toc = time.perf_counter()
print(f'The integral with N equal to 100000000 is {out} and takes {toc-tic} s to run')
print(f'The difference with the true value is {out - math.pi/2}')

The integral with N equal to 100000000 is 1.5707963267946878 and takes 58.6411332 s to run
The difference with the true value is -2.0872192862952943e-13
